<a href="https://colab.research.google.com/github/p-perrone/UiO_AdvancedRemoteSensing/blob/main/Project1_sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Import packages
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import random

In [16]:
# Run authentication
ee.Authenticate()
ee.Initialize(project='dulcet-iterator-470310-n0') # PUT YOUR API KEY (Project ID) HERE!

# Part 1: loading data

In [73]:
# this is for getting rid of edges (usually artifacted)
def mask_edge(image):
  edge = image.lt(-50.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

# testing geometry
svalbard_geom = ee.Geometry.Polygon(
    [[[2501471, 14525773],
      [2572610, 14525773],
      [2572610, 14566293],
      [2501471, 14566293],
      [2501471, 14525773]]],
    proj='EPSG:3857',
    geodesic=False
)


# Part 2: Defining Olga Strait geometry and filtering data

In [74]:
# Define Olga Strait geometry (replace with actual coordinates)
olgastretet_geom = ee.Geometry.Polygon(
    [[[0, 80], [30, 80], [30, 78], [0, 78], [0, 80]]]
)

Sentinel-1 does not operate in a single, fixed mode all the time. It switches between modes (like IW, EW, SM, WV) based on a pre-defined mission plan. These modes have different default polarization configurations.
* Interferometric Wide Swath (IW): The most common mode over land. It is typically programmed to acquire in VV+VH (most common) or HH+HV.
* Extra Wide Swath (EW): Used over large areas like oceans and sea ice. It is often programmed for HH+HV or sometimes VV+VH.
* Wave Mode (WV): Used for sampling ocean waves. It is almost always VV only.
* Stripmap Mode (SM): Offers higher resolution and can be programmed for single (HH or VV) or dual (HH+HV or VV+VH) polarization.

For this work, based on existing literature, we focus on HH, HV, HH+HV with EW.

In [75]:
polarizations = ['HH', 'HV']
svalbard_s1_dict = {}

In [76]:

# Sentinel-1 collection for Svalb

for pol in polarizations:
    svalbard_s1_dict[pol] = (

    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(svalbard_geom)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
    .filter(ee.Filter.eq('instrumentMode', 'EW'))
    .select(pol)
    .map(mask_edge)
    )

    # https://www.sciencedirect.com/science/article/pii/S0034425724002116?via%3Dihub


In [77]:
svalbard_s1_pol = [
svalbard_s1HH := svalbard_s1_dict['HH'],
svalbard_s1HV := svalbard_s1_dict['HV']]

In [78]:
# time ranges
spring = ee.Filter.date('2025-03-01', '2025-04-20')
late_spring = ee.Filter.date('2025-04-21', '2025-06-10')
summer = ee.Filter.date('2025-06-11', '2025-08-31')

seasons = [spring, late_spring, summer]

In [79]:
Map = geemap.Map()
Map.centerObject(olgastretet_geom, zoom=6)
Map.addLayer(svalbard_s1HH, {'min': -25, 'max': 5}, 'Sentinel-1') # display a certain range of dB
Map

Map(center=[79.33695198759294, 14.999999999999998], controls=(WidgetControl(options=['position', 'transparent_…

In [80]:
# building the Sentinel-1 polarizations-seasons dataframe
season_names = ['spring', 'late_spring', 'summer']

svalbard_s1 = pd.DataFrame(
    index=polarizations,
    columns=season_names,
)

for pol in polarizations:
    for i, season_filter in enumerate(seasons):

        filtered_collection = svalbard_s1_dict[pol].filter(season_filter)

        svalbard_s1.loc[pol, season_names[i]] = filtered_collection

display(svalbard_s1)

,spring,late_spring,summer
HH,"ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal..."
HV,"ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal...","ee.ImageCollection({\n ""functionInvocationVal..."


In [81]:
svalbard_s1_spring = [
svalbard_s1HH_spring := svalbard_s1.loc['HH', 'spring'],
svalbard_s1HV_spring := svalbard_s1.loc['HV', 'spring']]

svalbard_s1_latespring = [
svalbard_s1HH_late_spring := svalbard_s1.loc['HH', 'late_spring'],
svalbard_s1HV_late_spring := svalbard_s1.loc['HV', 'late_spring']]

svalbard_s1_summer = [
svalbard_s1HH_summer := svalbard_s1.loc['HH', 'summer'],
svalbard_s1HV_summer := svalbard_s1.loc['HV', 'summer']]

In [82]:
"GEMINI HELPED"

def dates_with_data(collection, geometry):
    """ Finds the dates in which layers of a collection exist in a given geometry.

        Parameters:
        :: collection = layers collection
        :: geometry   = area of interest

        Returns:
        :: dates = list of dates with data (as 'yyyy-MM-ddTHH:mm:ss' strings)
    """

    # filter the collection by geometry
    filtered_collection = collection.filterBounds(geometry)

    # get a list of all image dates in the filtered collection
    # system:time_start is an attribute of every image (= starting time of aquisition)
    image_dates = ee.List(filtered_collection.aggregate_array('system:time_start'))

    # Check if there are any images
    if image_dates.size().getInfo() > 0:
        # Convert timestamps to dates and format them as strings
        dates = image_dates.map(lambda time: ee.Date(time).format(None, 'GMT'))
        return dates.sort().getInfo()
    else:
        return None

# Find the dates with data for HH polarization in the spring collection
# Define the sea ice geometry using the provided diagonal vertices
# Coordinates are in EPSG:3857
sea_ice_geom = ee.Geometry.Polygon(
    [[[2501471, 14525773],
      [2572610, 14525773],
      [2572610, 14566293],
      [2501471, 14566293],
      [2501471, 14525773]]],
    proj='EPSG:3857',  # Specify the CRS here
    geodesic=False # Set to False for projected coordinates
)

dates_spring_HH = dates_with_data(svalbard_s1HH_spring, sea_ice_geom)
dates_spring_HV = dates_with_data(svalbard_s1HV_spring, sea_ice_geom)

# they are apparently the same
print(dates_spring_HH == dates_spring_HV)

dates_spring = dates_spring_HH
# Print the dates found
print(dates_spring)

True
['2025-03-01T05:50:27', '2025-03-02T06:31:37', '2025-03-03T05:34:02', '2025-03-05T05:17:33', '2025-03-06T05:58:42', '2025-03-08T05:42:13', '2025-03-09T06:23:23', '2025-03-10T05:25:47', '2025-03-11T06:06:57', '2025-03-13T05:50:28', '2025-03-14T06:31:37', '2025-03-15T05:34:02', '2025-03-17T05:17:33', '2025-03-18T05:58:42', '2025-03-20T05:42:13', '2025-03-21T06:23:23', '2025-03-22T05:25:48', '2025-03-23T06:06:57', '2025-03-25T05:50:28', '2025-03-26T06:31:37', '2025-03-27T05:34:02', '2025-03-28T05:24:44', '2025-03-29T05:17:33', '2025-03-29T06:05:44', '2025-03-30T05:58:43', '2025-04-01T05:42:14', '2025-04-02T06:23:23', '2025-04-03T05:25:48', '2025-04-04T06:06:57', '2025-04-06T05:50:28', '2025-04-07T06:31:38', '2025-04-08T05:34:02', '2025-04-10T05:17:33', '2025-04-10T06:05:45', '2025-04-11T05:58:42', '2025-04-12T05:49:22', '2025-04-13T05:42:13', '2025-04-14T06:23:23', '2025-04-15T05:25:47', '2025-04-16T06:06:57', '2025-04-17T05:57:34', '2025-04-18T05:50:28', '2025-04-19T06:31:38']


In [83]:

svalbard_s1_spring

By using only HH and HV we make sure to have images of the same location always at the same time. Probably the acquisition method of S1 has always been EW in the last thre years (?)

### Visualizing S1 data for spring

In [89]:
random.seed(35)
# Spring
SpringMap = geemap.Map()
SpringMap.centerObject(svalbard_geom, zoom=6)
spring_list = svalbard_s1HH_spring.toList(svalbard_s1HH_spring.size())
spring_s1_mosaics = []
spring_s1_day_mosaics = []

cmaps = ['bone', 'viridis']

# pick randome date. With seed = 3 we have a Mars day, good for sea ice classfct
if dates_spring:
        rnd_date_str = random.choice(dates_spring)
        rnd_date = ee.Date(rnd_date_str)

else:
        print("No HH data dates found in spring to select a random date from.")

# adding some layers
for cmap, ic, pol in zip(cmaps, svalbard_s1_spring, polarizations):
    # adding season mosaics (whole spring)
    ic_mosaic = ic.mosaic()
    spring_s1_mosaics.append(ic_mosaic)
    #SpringMap.addLayer(ic_mosaic, {'min': -25, 'max': 5}, f'{pol} Mosaic')

    # adding random daily mosaics
    # check if the date list is not empty

    ic_date = ic.filterDate(rnd_date, rnd_date.advance(1, 'day')) # Filter for the whole day

    # Mosaic the images for the selected date
    ic_day_mosaic = ic_date.mosaic()
    spring_s1_day_mosaics.append(ic_day_mosaic)

    # Add the mosaicked image from the random date to the map
    SpringMap.addLayer(ic_day_mosaic, {'min': -25, 'max': 5, 'palette': cmap}, f'{pol} on {rnd_date.format("YYYY-MM-dd").getInfo()}')

SpringMap

Map(center=[78.32673233318731, 22.790622575802143], controls=(WidgetControl(options=['position', 'transparent_…

In [93]:
drawn_features = SpringMap.draw_last_feature
drawn_rectangle = drawn_features

if drawn_rectangle:
    # Get the geometry of the feature
    rectangle_geometry = drawn_rectangle.geometry()

    # Get the coordinates of the geometry as a Python object
    coordinates = rectangle_geometry.getInfo()['coordinates']

    # Print the coordinates
    print("Coordinates of the drawn rectangle:")
    print(coordinates)
else:
    print("Cannot extract coordinates as no rectangle feature is available.")

Coordinates of the drawn rectangle:
[[[36.683029, 80.070371], [36.683029, 80.390982], [40.747361, 80.390982], [40.747361, 80.070371], [36.683029, 80.070371]]]


In [94]:
# this is a good area to see a bit how the radar performs with sea ice. In the selected date
# (seed = 35, 2025-04-12), there is some bare ice, different ice texturess, snow on ice and deep water
test_geom = ee.Geometry.Polygon([[36.683029, 80.070371], [36.683029, 80.390982], [40.747361, 80.390982], [40.747361, 80.070371], [36.683029, 80.070371]])

In [30]:
drawn_features = SpringMap.draw_last_feature
drawn_rectangle = drawn_features

if drawn_rectangle:
    # Get the geometry of the feature
    rectangle_geometry = drawn_rectangle.geometry()

    # Get the coordinates of the geometry as a Python object
    coordinates = rectangle_geometry.getInfo()['coordinates']

    # Print the coordinates
    print("Coordinates of the drawn rectangle:")
    print(coordinates)
else:
    print("Cannot extract coordinates as no rectangle feature is available.")

sea_ice_geom2 = ee.Geometry.Polygon(
    coordinates,
    geodesic=False # Set to False for projected coordinates
)
# Test the sample_image_collection function with an ImageCollection
# Using svalbard_s1HH_spring and random_points
sampled_spring_hh = sample_image_collection(svalbard_s1HH_spring.filterBounds(sea_ice_geom2), random_points)

# Print the sampled data (note: this will only show a small sample in the console)
# To get all data, you would typically export this FeatureCollection
print("Sampled data from svalbard_s1HH_spring (first 10 features):")
print(sampled_spring_hh.limit(10).getInfo())

Cannot extract coordinates as no rectangle feature is available.
Sampled data from svalbard_s1HH_spring (first 10 features):
{'type': 'FeatureCollection', 'columns': {}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [27.679549103639907, 79.100209640881]}, 'id': 'S1A_EW_GRDM_1SDH_20230302T053404_20230302T053508_047461_05B2AE_29FD_0', 'properties': {'first': -14.430913735366675, 'timestamp': 1677735244000}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [28.09330257339795, 79.11255566018686]}, 'id': 'S1A_EW_GRDM_1SDH_20230302T053404_20230302T053508_047461_05B2AE_29FD_1', 'properties': {'first': -14.088480276579453, 'timestamp': 1677735244000}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [27.690457142748443, 79.05772246748712]}, 'id': 'S1A_EW_GRDM_1SDH_20230302T053404_20230302T053508_047461_05B2AE_29FD_2', 'properties': {'first': -14.461108871593149, 'timestamp': 1677735244000}}, {'type': 'Feature', 'geometry': {'typ

In [31]:
# Define the number of random points to generate initially (significantly more than needed)
initial_num_points = 1000

# Generate random points within the bounds of the sea ice geometry
random_points = ee.FeatureCollection.randomPoints(sea_ice_geom2, initial_num_points,
    seed=42)

In [32]:
# Define the number of random points to generate initially (significantly more than needed)
initial_num_points = 1000

# Generate random points within the bounds of the sea ice geometry
random_points = ee.FeatureCollection.randomPoints(sea_ice_geom2, initial_num_points,
    seed=42)

# Convert the sampled data to a pandas DataFrame
sampled_df = geemap.ee_to_df(sampled_spring_hh)

# Filter out rows with missing values (NaNs) that might result from sampling no-data areas
sampled_df = sampled_df.dropna()

# Display the first few rows of the DataFrame
print(sampled_df)

# Print the sampled data (note: this will only show a small sample in the console)
# To get all data, you would typically export this FeatureCollection
# print(sampled_data.limit(10).getInfo()) # This line is no longer needed

SpringMap.addLayer(random_points, {'color': '00FF00'}, 'Sampled Points')
SpringMap.centerObject(sea_ice_geom2, 9) # Center map on the image

SpringMap

           first      timestamp
0     -14.430914  1677735244000
1     -14.088480  1677735244000
2     -14.461109  1677735244000
3     -12.560343  1677735244000
4     -11.134371  1677735244000
...          ...            ...
31995 -13.934731  1681882444000
31996 -11.920331  1681882444000
31997 -13.497951  1681882444000
31998 -15.475178  1681882444000
31999 -14.263978  1681882444000

[30455 rows x 2 columns]


Map(bottom=16873.0, center=[79.07349152688319, 27.948963499999937], controls=(WidgetControl(options=['position…

In [31]:
# Check if the geometry intersects with the image bounds
intersects = sea_ice_geom2.intersects(spring_image_hh.geometry(), ee.ErrorMargin(1))
print(f"Does the geometry intersect the image? {intersects.getInfo()}")

# Check the number of images in the collection after filtering and masking
collection_size = svalbard_s1HH_spring.filterBounds(sea_ice_geom2).size()
print(f"Number of images in the filtered collection: {collection_size.getInfo()}")

# Check if the first image in the filtered collection has any unmasked pixels within the geometry
# This requires reducing the image over the geometry
if collection_size.getInfo() > 0:
    first_image_filtered = ee.Image(svalbard_s1HH_spring.filterBounds(sea_ice_geom2).first())
    # Use a simple reducer to check for non-masked pixels (e.g., count)
    count_reducer = first_image_filtered.select('HH').reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=sea_ice_geom2,
        scale=20,  # Use the same scale as sampling
        maxPixels=1e9 # Increase maxPixels for potentially large geometries
    )
    print(f"Count of non-masked pixels in the first filtered image within the geometry: {count_reducer.getInfo()}")

Does the geometry intersect the image? False
Number of images in the filtered collection: 32
Count of non-masked pixels in the first filtered image within the geometry: {'HH': 1191846}
